In [228]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the time module.
import time

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

%matplotlib inline

In [229]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [230]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [231]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [232]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

24.37405344710686 -159.81565313467368
20.207828956859146 2.99703000118879
84.65423854884497 143.80040365033415
-77.8489415815381 -137.49163667442483
-72.55556513648365 50.521810387971726


In [233]:
# Use the citipy module to determine city based on latitude and longitude - Generate citie
from citipy import citipy

In [234]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

kapaa us
tessalit ml
chokurdakh ru
rikitea pf
east london za


In [235]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
print(cities)


['kapaa', 'tessalit', 'chokurdakh', 'rikitea', 'east london']


In [236]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [240]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        print(city_weather)
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        weather_description = city_weather["weather"][0]['description']
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather Description": weather_description
                          
                          #"Weather Description":city_weather_description})
                         })
# If an error is experienced, skip the city.
    except IndexError:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    
    
    
    

Processing Record 6 of Set 1 | kapaa
{'coord': {'lon': -159.319, 'lat': 22.0752}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 69.22, 'feels_like': 68.85, 'temp_min': 66.99, 'temp_max': 73.4, 'pressure': 1018, 'humidity': 64}, 'visibility': 10000, 'wind': {'speed': 19.57, 'deg': 40}, 'clouds': {'all': 40}, 'dt': 1618378068, 'sys': {'type': 1, 'id': 7873, 'country': 'US', 'sunrise': 1618330716, 'sunset': 1618376177}, 'timezone': -36000, 'id': 5848280, 'name': 'Kapaa', 'cod': 200}
Processing Record 7 of Set 1 | tessalit
{'coord': {'lon': 1.0114, 'lat': 20.1986}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 83.77, 'feels_like': 80.92, 'temp_min': 83.77, 'temp_max': 83.77, 'pressure': 1009, 'humidity': 11, 'sea_level': 1009, 'grnd_level': 952}, 'visibility': 10000, 'wind': {'speed': 7.09, 'deg': 121, 'gust': 7.23}, 'clouds': {

In [238]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,Kapaa,22.0752,-159.3190,73.40,64,40,19.57,scattered clouds
1,Tessalit,20.1986,1.0114,83.77,11,23,7.09,few clouds
2,Chokurdakh,70.6333,147.9167,12.18,82,16,5.26,few clouds
3,Rikitea,-23.1203,-134.9692,77.77,66,5,8.39,clear sky
4,East London,-33.0153,27.9116,69.80,94,40,11.50,scattered clouds


In [239]:
# Create the output file (CSV).
output_data_file = "weather_database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

